# SpaceX  Falcon 9 First Stage Landing Prediction
## Part 2: Web Scraping Launch Records from Wikipedia
***
### Objectives
*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame
***
### Import Libraries and Define Auxiliary Functions

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

Let's define a series of functions that will help us proess the web-scraped HTML table

In [3]:
def date_time(table_cells):
    """
    Returns date and time from HTML table cell
    """
    return [date_time.strip() for date_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Returns booster version from HTML table cell
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    Returns landing status from HTML table celll 
    """
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """
    Returns payloaf mass from HTML table celll 
    """
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    """
    Returns column headers from HTML table
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

### Data Acquisition

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

# Check if the request was successful (success = code 200)
response.status_code

200

In [5]:
# Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Check title to verify the BeautifulSoup object was created properly
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Extract all column/variable names from the HTML table header.


In [6]:
html_tables = soup.find_all('table')

# Extract the third table (contains the actual launch records)
first_launch_table = html_tables[2]

Next, we need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one.

In [7]:
column_names = []
first_launch_table_th = first_launch_table.find_all('th')

for th in first_launch_table_th:
    column_name = extract_column_from_header(th)
    if column_name is not None and len(str(column_name)) >0:
        column_names.append(column_name)
        
# Check the extracted column names
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Now we can create a dataframe by parsing the launch HTML tables

In [8]:
# Create empty dictionary from extracted column names
launch_dict= dict.fromkeys(column_names)

# Remove 'Date and time ( )' column (we will replace with separate Date and Time columns)
del launch_dict['Date and time ( )']

# Initiate launch_dict with each column to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Add some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we need to fill up the `launch_dict` with launch records extracted from table rows and then convert the dictionary to a pandas dataframe.

In [9]:
# Fill launch_dict with launch records
extracted_row = 0
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date Time
            datatimelist=date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            time = datatimelist[1]
            launch_dict['Time'].append(time)
             
            # Booster Version    
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            href_in = row[6].find_all('a')
            if href_in:
                customer = row[6].a.string
                launch_dict['Customer'].append(customer)
            else:
                customer = row[6].string.strip('\n')
                launch_dict['Customer'].append(customer)
            
            # Launch Outcome
            launch_outcome = list(row[7].strings)[0].strip('\n')
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster Landing
            booster_landing = landing_status(row[8]).strip('\n')
            launch_dict['Booster landing'].append(booster_landing)
            
# Convert launch_dict to a pandas dataframe
df=pd.DataFrame(launch_dict)
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.0B0007.1,No attempt,1 March 2013,15:10


In [10]:
# Drop rows with Launch Site "Cape Canaveral"
df.drop(df[df['Launch site'] == 'Cape Canaveral'].index, inplace=True)

Lastly, we can export the data to a <b>CSV</b>.

In [11]:
df.to_csv('02_dataset.csv', index=False)